In [ ]:
# 필수 라이브러리 임포트
import torch
import json
import os
import numpy as np
from typing import List, Dict, Any

# Hugging Face 관련 라이브러리
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,      # Causal Language Model 로드
    AutoTokenizer,              # 토크나이저 로드
    BitsAndBytesConfig,         # 양자화 설정
    TrainingArguments,
    TrainerCallback
)

# TRL: Transformer Reinforcement Learning 라이브러리
from trl import GRPOConfig, GRPOTrainer

# PEFT: Parameter-Efficient Fine-Tuning
from peft import (
    LoraConfig,                          # LoRA 설정
    get_peft_model,                      # 모델에 PEFT 적용
    prepare_model_for_kbit_training,     # 양자화 모델 학습 준비
    PeftModel
)

# GPU 정보 확인
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")

# 랜덤 시드 고정 (재현성 확보)
SEED = 1210 # 내 생일 ㅎㅎ
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("✅ 라이브러리 임포트 완료")

In [ ]:
from datasets import load_from_disk

# 데이터 경로 설정
TRAIN_DATA_PATH = "./training_dataset"  # 저장된 학습 데이터
TEST_DATA_PATH = "./testing_dataset"    # 저장된 평가 데이터

print("📂 저장된 데이터셋 로딩 중...")

# 디스크에서 데이터셋 로드
# 이미 Dataset.save_to_disk()로 저장된 형식이므로 빠르게 로드 가능
try:
    train_dataset = load_from_disk(TRAIN_DATA_PATH)
    eval_dataset = load_from_disk(TEST_DATA_PATH)
    
    print(f"✅ 학습 데이터: {len(train_dataset)} 샘플")
    print(f"✅ 평가 데이터: {len(eval_dataset)} 샘플")
    
    # 데이터 샘플 확인
    print("\n📝 데이터 샘플 (첫 번째):")
    print(f"Query (처음 300자):\n{train_dataset[0]['query'][:300]}...")
    print(f"\nAnswer: {train_dataset[0]['answer']}")
    
    # 데이터 형식 확인
    print("\n🔍 데이터 컬럼:", train_dataset.column_names)
    print("✅ 데이터 형식 확인 완료")
    
    # 데이터셋 구조 체크
    assert 'query' in train_dataset.column_names, "❌ 'query' 컬럼이 없습니다!"
    assert 'answer' in train_dataset.column_names, "❌ 'answer' 컬럼이 없습니다!"
    
except FileNotFoundError:
    print("❌ 데이터셋 파일을 찾을 수 없습니다!")
    print(f"   다음 경로를 확인하세요:")
    print(f"   - {TRAIN_DATA_PATH}")
    print(f"   - {TEST_DATA_PATH}")
    print("\n💡 데이터셋을 먼저 생성해야 합니다.")
    raise

# 프롬프트 형식 확인
print("\n📋 프롬프트 형식 분석:")
sample_query = train_dataset[0]['query']
has_think_tag = '<think>' in sample_query
has_answer_tag = '<answer>' in sample_query
print(f"  - <think> 태그 포함: {'✅' if has_think_tag else '❌'}")
print(f"  - <answer> 태그 포함: {'✅' if has_answer_tag else '❌'}")

if has_think_tag and has_answer_tag:
    print("✅ 추론 + 정답 형식의 프롬프트입니다.")
else:
    print("ℹ️  일반 형식의 프롬프트입니다.")

In [ ]:
# 모델 설정
MODEL_NAME = "Qwen/Qwen3-8B"  # 사용할 모델명

# 양자화 설정 (메모리 효율성을 위한 4bit 양자화)
# BitsAndBytes를 사용한 NF4(Normal Float 4) 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # 4bit 양자화 활성화
    bnb_4bit_quant_type="nf4",            # NF4 양자화 타입 (일반적으로 최고 성능)
    bnb_4bit_compute_dtype=torch.bfloat16, # 계산에 사용할 데이터 타입
    bnb_4bit_use_double_quant=True,       # 이중 양자화로 메모리 추가 절감
)

print("🔄 모델 로딩 중... (수 분 소요될 수 있습니다)")

# 모델 로드
# device_map="auto": 자동으로 여러 GPU에 모델 분산 배치
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,      # 양자화 설정 적용
    device_map="auto",                    # GPU에 자동 배치
    trust_remote_code=True,               # 커스텀 코드 실행 허용 (Qwen 모델)
    torch_dtype=torch.bfloat16,           # 메모리 효율적인 bfloat16 사용
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="left",  # GRPO는 left padding 권장 (생성 시 일관성)
)

# 패딩 토큰 설정
# Qwen 모델은 기본적으로 pad_token이 없으므로 eos_token 사용
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id
    print(f"⚙️  패딩 토큰 설정: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

# 모델 정보 출력
print(f"\n✅ 모델 로드 완료: {MODEL_NAME}")
print(f"📊 모델 파라미터 수: {model.num_parameters() / 1e9:.2f}B")
print(f"💾 모델 메모리 사용량: {model.get_memory_footprint() / 1e9:.2f} GB")

# 토크나이저 정보
print(f"\n📝 토크나이저 정보:")
print(f"  Vocab size: {tokenizer.vocab_size}")
print(f"  Padding side: {tokenizer.padding_side}")
print(f"  Pad token: {tokenizer.pad_token}")

In [ ]:
# LoRA(Low-Rank Adaptation) 설정
# LoRA는 전체 모델을 학습하는 대신 작은 행렬만 학습하여 메모리와 시간을 절약

lora_config = LoraConfig(
    # LoRA 하이퍼파라미터
    r=32,                      # Low-rank 행렬의 rank (8, 16, 32가 일반적)
                               # 작을수록 파라미터 적고 빠름, 클수록 표현력 증가
    
    lora_alpha=64,             # LoRA scaling factor (일반적으로 r의 2배)
                               # 학습률과 관련된 스케일링 파라미터
    
    lora_dropout=0.05,         # Dropout 비율 (과적합 방지)
    
    bias="none",               # Bias 학습 여부 ("none", "all", "lora_only")
    
    task_type="CAUSAL_LM",     # 태스크 타입 (Causal Language Modeling)
    
    # LoRA를 적용할 타겟 모듈들
    # Qwen 모델의 Attention과 FFN 레이어의 주요 투영 행렬들
    target_modules=[
        "q_proj",              # Query projection
        "k_proj",              # Key projection  
        "v_proj",              # Value projection
        "o_proj",              # Output projection
        "gate_proj",           # FFN gate projection
        "up_proj",             # FFN up projection
        "down_proj",           # FFN down projection
    ],
    
    # 추가 설정
    inference_mode=False,      # 학습 모드
)

print("🔧 LoRA 설정:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Target modules: {len(lora_config.target_modules)} layers")

# 양자화된 모델을 학습 가능하도록 준비
# gradient checkpointing 등을 자동으로 설정
model = prepare_model_for_kbit_training(model)
print("✅ 모델을 양자화 학습 모드로 전환 완료")

# 모델에 LoRA 어댑터 추가
model = get_peft_model(model, lora_config)
print("✅ LoRA 어댑터 적용 완료")

# 학습 가능한 파라미터 정보 출력
model.print_trainable_parameters()
# 출력 예시:
# trainable params: 20M || all params: 8B || trainable%: 0.25%
# → 전체 모델의 0.25%만 학습하므로 매우 효율적

In [ ]:
import re

def extract_answer_from_response(response: str) -> str:
    """
    모델 응답에서 답변 번호 추출
    
    사용자가 설정한 형식: <think>추론</think> <answer>숫자</answer>
    
    Args:
        response: 모델이 생성한 텍스트
    
    Returns:
        추출된 답변 번호 (1-5) 또는 빈 문자열
    """
    # 패턴 1: <answer> 태그 안의 숫자 (최우선)
    # 예: <answer>2</answer> 또는 <answer> 2 </answer>
    pattern_answer_tag = r'<answer>\s*([1-5])\s*</answer>'
    match = re.search(pattern_answer_tag, response, re.IGNORECASE)
    if match:
        return match.group(1)
    
    return ""


def compute_single_reward(query: str, response: str, ground_truth: str) -> float:
    """
    단일 샘플에 대한 보상 계산
    
    GRPO의 핵심: 응답의 품질을 정량화하는 보상 함수
    
    Args:
        query: 입력 질문
        response: 모델이 생성한 응답
        ground_truth: 정답
    
    Returns:
        보상 점수 (float)
        - 1.0: 정답
        - -0.5: 오답
        - -1.0: 형식 오류 (답변 추출 실패)
    
    Note:
        보상 함수 설계는 GRPO 성능에 결정적 영향을 미칩니다.
        - 너무 sparse하면 학습이 어려움
        - 너무 dense하면 overfitting 위험
    """
    # 응답에서 답변 추출
    predicted = extract_answer_from_response(response)
    
    # 형식 오류: 답변을 추출할 수 없음
    if not predicted:
        return -1.0
    
    # 정답 여부 확인
    if predicted == ground_truth:
        # 정답: 높은 양의 보상
        reward = 1.0
        
    else:
        # 오답: 음의 보상 (하지만 형식 오류보다는 덜 나쁨)
        reward = -0.5
    
    return reward


def reward_function(samples: List[str], prompts: List[str], 
                   ground_truths: List[str], **kwargs) -> List[float]:
    """
    배치 단위 보상 계산 함수 (GRPO Trainer에서 호출)
    
    Args:
        samples: 모델이 생성한 응답들의 리스트
        prompts: 입력 프롬프트들의 리스트
        ground_truths: 정답 레이블들의 리스트
        **kwargs: 추가 인자들 (사용하지 않음)
    
    Returns:
        각 샘플에 대한 보상 점수 리스트
    
    Note:
        GRPO는 그룹 내 상대적 보상을 사용합니다.
        같은 쿼리에 대한 여러 응답을 비교하여 최적화합니다.
    """
    rewards = []
    
    for sample, prompt, gt in zip(samples, prompts, ground_truths):
        reward = compute_single_reward(prompt, sample, gt)
        rewards.append(reward)
    
    return rewards


# Reward Function 테스트
print("🧪 Reward Function 테스트 (사용자 형식 기준):")

test_cases = [
    ("test query", "<think>이 문제는 지문의 주제를 파악하는 문제입니다...</think><answer>2</answer>", "2", "완벽한 형식 - 정답"),
    ("test query", "<think>분석 과정...</think> <answer>3</answer>", "3", "공백 포함 - 정답"),
    ("test query", "<think>추론...</think><answer>2</answer>", "1", "완벽한 형식 - 오답"),
    ("test query", "<answer>4</answer>", "4", "think 없이 answer만 - 정답"),
    ("test query", "<think>고민 중...</think>답은 5번입니다", "5", "태그 없이 텍스트 - 정답"),
    ("test query", "<think>추론 중...</think><answer>2", "2", "answer 태그 미완성"),
    ("test query", "2번이 정답입니다", "2", "태그 없는 일반 답변"),
    ("test query", "음... 어려운 문제네요", "1", "형식 오류"),
]

print("\n" + "="*70)
for query, response, gt, desc in test_cases:
    reward = compute_single_reward(query, response, gt)
    extracted = extract_answer_from_response(response)
    status = "✅" if extracted == gt else "❌"
    print(f"{status} {desc:30s} | 추출: {extracted:1s} | 정답: {gt} | 보상: {reward:+.2f}")

print("="*70)
print("✅ Reward Function 준비 완료")
print("\n💡 Tips:")
print("  - <answer> 태그 안의 숫자가 최우선으로 추출됩니다")
print("  - 태그가 없어도 여러 패턴으로 숫자 추출을 시도합니다")
print("  - 추론 과정(<think>)은 보상 계산에 직접 영향을 주지 않습니다")

In [ ]:
# 출력 디렉토리 설정
OUTPUT_DIR = "./grpo_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# GRPO 학습 설정
# GRPOConfig는 TrainingArguments를 상속하며 GRPO 특화 파라미터를 추가
training_args = GRPOConfig(
    # ===== 기본 설정 =====
    output_dir=OUTPUT_DIR,
    run_name="qwen_gpro_korean_ksat",
    
    # ===== 학습 기본 파라미터 ===== 
    num_train_epochs=3,
    per_device_train_batch_size=4,          # 1 → 4 (4배 증가)
    per_device_eval_batch_size=4,           # 1 → 4
    gradient_accumulation_steps=8,          # 16 → 8 (실질적 배치: 4*8*3 = 96)
    
    # ===== 옵티마이저 설정 =====
    learning_rate=5e-6,                     # 배치 크기 증가 시 learning_rate도 약간 증가 고려 (7e-6 정도)
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=1.0,
    
    # ===== GRPO 특화 파라미터 =====
    max_length=10000,
    max_prompt_length=5000,
    max_completion_length=5000,
    
    # 생성 설정
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    
    # GRPO 핵심 파라미터
    num_generations=8,                      # 4 → 8 (더 많은 샘플 비교로 성능 향상)
    
    # KL Divergence 제약
    kl_coef=0.05,
    
    # ===== 로깅 및 체크포인트 =====
    logging_steps=5,                        # 10 → 5 (더 자주 로깅)
    save_steps=50,                          # 100 → 50
    eval_steps=50,                          # 100 → 50
    save_total_limit=3,
    
    # ===== 평가 설정 =====
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval/reward_mean",
    greater_is_better=True,
    
    # ===== 하드웨어 최적화 =====
    bf16=True,
    gradient_checkpointing=True,
    dataloader_num_workers=8,               # 4 → 8 (데이터 로딩 병렬화 증가)
    
    # ===== 분산 학습 (Multi-GPU) =====
    ddp_find_unused_parameters=False,
    
    # ===== 기타 =====
    seed=SEED,
    report_to="tensorboard",
)


print("⚙️  GRPO 학습 설정:")
print(f"  실질적 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps * torch.cuda.device_count()}")
print(f"  총 학습 스텝 수: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps * torch.cuda.device_count()) * training_args.num_train_epochs}")
print(f"  각 쿼리당 생성 샘플: {training_args.num_generations}")
print(f"  KL Divergence 계수: {training_args.kl_coef}")

In [ ]:
# 커스텀 콜백: 학습 과정 모니터링
import datetime


class GRPOMonitorCallback(TrainerCallback):
    """
    GRPO 학습의 주요 메트릭을 모니터링하는 콜백
    """
    def __init__(self):
        self.best_reward = float('-inf')
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        """
        로깅 발생 시 호출
        """
        if logs is None:
            return
        
        # 학습 메트릭 출력
        if 'train/reward_mean' in logs:
            reward = logs['train/reward_mean']
            kl = logs.get('train/kl', 0)
            loss = logs.get('loss', 0)
            
            print(f"\n📊 Step {state.global_step}:")
            print(f"  ├─ Reward Mean: {reward:.4f}")
            print(f"  ├─ KL Divergence: {kl:.6f}")
            print(f"  └─ Loss: {loss:.4f}")
            
            # 최고 보상 업데이트
            if reward > self.best_reward:
                self.best_reward = reward
                print(f"  ⭐ New best reward: {self.best_reward:.4f}")
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """
        평가 완료 시 호출
        """
        if metrics is None:
            return
        
        eval_reward = metrics.get('eval/reward_mean', 0)
        print(f"\n📈 Evaluation at step {state.global_step}:")
        print(f"  └─ Eval Reward Mean: {eval_reward:.4f}")


# GRPO Trainer 초기화
print("\n🚀 GRPO Trainer 초기화 중...")

trainer = GRPOTrainer(
    model=model,                            # 학습할 모델
    args=training_args,                     # 학습 설정
    train_dataset=train_dataset,            # 학습 데이터
    eval_dataset=eval_dataset,              # 평가 데이터
    tokenizer=tokenizer,                    # 토크나이저
    reward_fn=reward_function,              # 보상 함수
    peft_config=lora_config,                # LoRA 설정
    callbacks=[GRPOMonitorCallback()],      # 커스텀 콜백
)

print("✅ Trainer 초기화 완료")

# 학습 시작
print("\n" + "="*60)
print("🎓 GRPO 학습 시작")
print("="*60)
print(f"📅 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

try:
    # 학습 실행
    train_result = trainer.train()
    
    print("\n" + "="*60)
    print("✅ 학습 완료!")
    print("="*60)
    
    # 학습 결과 출력
    print("\n📊 최종 학습 결과:")
    print(f"  ├─ Total training time: {train_result.metrics.get('train_runtime', 0):.2f} seconds")
    print(f"  ├─ Training samples/second: {train_result.metrics.get('train_samples_per_second', 0):.2f}")
    print(f"  └─ Final loss: {train_result.metrics.get('train_loss', 0):.4f}")
    
    # 최종 모델 저장
    final_model_path = os.path.join(OUTPUT_DIR, "final_model")
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    
    print(f"\n💾 최종 모델 저장 완료: {final_model_path}")
    
except KeyboardInterrupt:
    print("\n⚠️  학습이 사용자에 의해 중단되었습니다.")
    print("현재까지의 체크포인트가 저장되었습니다.")
    
except Exception as e:
    print(f"\n❌ 학습 중 오류 발생: {str(e)}")
    raise

# TensorBoard 로그 확인 안내
print("\n📊 TensorBoard로 학습 과정 확인:")
print(f"  터미널에서 실행: tensorboard --logdir {OUTPUT_DIR}")

In [ ]:
from datetime import datetime

def generate_answer(
    model, 
    tokenizer, 
    query: str, 
    max_new_tokens: int = 5000,
    temperature: float = 0.7,
    top_p: float = 0.9,
    do_sample: bool = True
) -> str:
    """
    학습된 모델로 답변 생성
    
    Args:
        model: 학습된 모델
        tokenizer: 토크나이저
        query: 입력 질문
        max_new_tokens: 생성할 최대 토큰 수
        temperature: 샘플링 온도
        top_p: Nucleus sampling threshold
        do_sample: 샘플링 사용 여부 (False면 greedy decoding)
    
    Returns:
        생성된 응답 텍스트
    """
    # 모델을 평가 모드로 설정
    model.eval()
    
    # 입력 토크나이징
    inputs = tokenizer(
        query,
        return_tensors="pt",
        truncation=True,
        max_length=5000
    )
    
    # GPU로 이동
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=50,
            do_sample=do_sample,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 생성된 부분만 디코딩 (입력 프롬프트 제외)
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()


def evaluate_model_on_dataset(
    model,
    tokenizer,
    dataset: Dataset,
    num_samples: int = None,
    verbose: bool = True
) -> Dict[str, Any]:
    """
    데이터셋에 대한 모델 성능 평가
    
    Args:
        model: 평가할 모델
        tokenizer: 토크나이저
        dataset: 평가 데이터셋
        num_samples: 평가할 샘플 수 (None이면 전체)
        verbose: 상세 출력 여부
    
    Returns:
        평가 메트릭 딕셔너리
    """
    if num_samples is None:
        num_samples = len(dataset)
    
    correct = 0
    total = 0
    format_errors = 0
    
    results = []
    
    print(f"\n🧪 모델 평가 시작 ({num_samples} 샘플)")
    print("="*60)
    
    for i in range(min(num_samples, len(dataset))):
        sample = dataset[i]
        query = sample['query']
        ground_truth = sample['answer']
        
        # 답변 생성
        response = generate_answer(model, tokenizer, query)
        
        # 답변 추출
        predicted = extract_answer_from_response(response)
        
        # 평가
        if not predicted:
            format_errors += 1
            is_correct = False
        else:
            is_correct = (predicted == ground_truth)
            if is_correct:
                correct += 1
        
        total += 1
        
        # 결과 저장
        results.append({
            'query': query,
            'response': response,
            'predicted': predicted,
            'ground_truth': ground_truth,
            'correct': is_correct
        })
        
        # 상세 출력
        if verbose and i < 5:  # 처음 5개만 출력
            print(f"\n샘플 {i+1}:")
            print(f"  Query (처음 100자): {query[:100]}...")
            print(f"  Response: {response}")
            print(f"  Predicted: {predicted}")
            print(f"  Ground Truth: {ground_truth}")
            print(f"  Correct: {'✅' if is_correct else '❌'}")
    
    # 메트릭 계산
    accuracy = correct / total if total > 0 else 0
    format_error_rate = format_errors / total if total > 0 else 0
    
    metrics = {
        'accuracy': accuracy,
        'correct': correct,
        'total': total,
        'format_errors': format_errors,
        'format_error_rate': format_error_rate
    }
    
    print("\n" + "="*60)
    print("📊 평가 결과:")
    print(f"  ├─ Accuracy: {accuracy*100:.2f}%")
    print(f"  ├─ Correct: {correct}/{total}")
    print(f"  └─ Format Errors: {format_errors} ({format_error_rate*100:.2f}%)")
    print("="*60)
    
    return metrics, results


# ===== 테스트 샘플로 빠른 확인 =====
print("\n🎯 단일 샘플 테스트:")
test_sample = eval_dataset[0]
test_response = generate_answer(model, tokenizer, test_sample['query'])

print(f"Query (처음 300자):\n{test_sample['query'][:300]}...\n")
print(f"="*70)
print(f"Generated Response:\n{test_response}\n")
print(f"="*70)
print(f"Ground Truth: {test_sample['answer']}")

# 답변 추출 및 평가
predicted = extract_answer_from_response(test_response)
print(f"Predicted: {predicted}")

# 형식 체크
has_think = '<think>' in test_response.lower()
has_answer = '<answer>' in test_response.lower()
print(f"\n형식 확인:")
print(f"  - <think> 태그 사용: {'✅' if has_think else '❌'}")
print(f"  - <answer> 태그 사용: {'✅' if has_answer else '❌'}")

# 정답 여부
is_correct = predicted == test_sample['answer']
print(f"\n결과: {'✅ Correct!' if is_correct else '❌ Wrong'}")

if not is_correct:
    print(f"💡 모델이 {predicted}번을 선택했지만 정답은 {test_sample['answer']}번입니다.")


# ===== 전체 평가 데이터셋 평가 =====
print("\n" + "="*60)
eval_metrics, eval_results = evaluate_model_on_dataset(
    model=model,
    tokenizer=tokenizer,
    dataset=eval_dataset,
    num_samples=None,  # 전체 평가
    verbose=False  # 샘플별 출력 비활성화
)

# 결과 저장
results_save_path = os.path.join(OUTPUT_DIR, "evaluation_results.json")
with open(results_save_path, 'w', encoding='utf-8') as f:
    json.dump({
        'metrics': eval_metrics,
        'results': eval_results[:10]  # 처음 10개만 저장
    }, f, ensure_ascii=False, indent=2)

print(f"\n💾 평가 결과 저장: {results_save_path}")


# ===== 대화형 테스트 =====
print("\n" + "="*60)
print("💬 대화형 테스트 모드")
print("사용법: 독해 문제를 입력하면 모델이 답변을 생성합니다.")
print("       (프롬프트 형식은 자동으로 적용되지 않습니다)")
print("종료: 'quit' 또는 'exit' 입력")
print("="*60)

def interactive_test():
    """
    대화형 테스트 함수
    
    Note:
        사용자가 이미 프롬프트 형식을 만들었으므로
        추가 포맷팅 없이 그대로 사용합니다.
    """
    while True:
        print("\n질문을 입력하세요 (또는 'sample'로 샘플 실행):")
        user_input = input("> ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("종료합니다.")
            break
        
        if not user_input:
            continue
        
        # 샘플 데이터 사용
        if user_input.lower() == 'sample':
            if len(eval_dataset) > 0:
                import random
                sample_idx = random.randint(0, len(eval_dataset) - 1)
                user_input = eval_dataset[sample_idx]['query']
                print(f"\n[샘플 #{sample_idx}]")
                print(f"Ground Truth: {eval_dataset[sample_idx]['answer']}")
            else:
                print("❌ 평가 데이터셋이 비어있습니다.")
                continue
        
        # 답변 생성
        print("\n🤔 생성 중...")
        response = generate_answer(model, tokenizer, user_input)
        
        print(f"\n📝 모델 답변:\n{response}")
        
        # 답변 추출
        predicted = extract_answer_from_response(response)
        if predicted:
            print(f"\n✅ 추출된 답변: {predicted}번")
        else:
            print("\n⚠️  답변을 추출할 수 없습니다.")

# 대화형 모드 실행 (선택사항)
# 주석 해제하여 실행:
# interactive_test()